<a href="https://colab.research.google.com/github/udkandi/Wordle-Solver/blob/main/Code_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/submit.py')

In [ ]:
from submit import my_fit

In [ ]:
import time as tm
import pickle
import warnings
import os

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
with open( "dict", 'r' ) as f:
	words = f.read().split( '\n' )[:-1]		# Omit the last line since it is empty
	num_words = len(words)

In [ ]:
class Merlin:
	def __init__( self, query_max, words ):
		self.words = words
		self.num_words = len( words )
		self.secret = ""
		self.query_max = query_max
		self.arthur = None
		self.win_count = 0
		self.tot_query_count = 0
		self.rnd_query_count = 0

	def meet( self, arthur ):
		self.arthur = arthur

	def reset( self, secret ):
		self.secret = secret
		self.rnd_query_count = 0

	# Receive a message from Arthur
	# Process it and terminate the round or else message Arthur back
	# Arthur can set is_done to request termination of this round after this query
	def msg( self, query_idx, is_done = False ):

		# Supplying an illegal value for query_idx is a way for Arthur to request
		# termination of this round immediately without even processing the current query
		# However, this results in query count being set to max for this round
		if query_idx < 0 or query_idx > self.num_words - 1:
			warnings.warn( "Warning: Arthur has sent an illegal query -- terminating this round", UserWarning )
			self.tot_query_count += self.query_max
			return

		# Arthur has made a valid query
		# Find the guessed word and increase the query counter
		query = self.words[ query_idx ]
		self.rnd_query_count += 1

		# Find out the intersections between the query and the secret
		reveal = [ *( '_' * len( self.secret ) ) ]

		for i in range( min( len( self.secret ), len( query ) ) ):
			if self.secret[i] == query[i]:
				reveal[ i ] = self.secret[i]

		# The word was correctly guessed
		if '_' not in reveal:
			self.win_count += 1
			self.tot_query_count += self.rnd_query_count
			return

		# Too many queries have been made - terminate the round
		if self.rnd_query_count >= self.query_max:
			self.tot_query_count += self.rnd_query_count
			return

		# If Arthur is done playing, terminate this round
		if is_done:
			self.tot_query_count += self.rnd_query_count
			return

		# If none of the above happen, continue playing
		self.arthur.msg( ' '.join( reveal ) )

	def reset_and_play( self, secret ):
		self.reset( secret )
		self.arthur.msg( ( "_ " * len( self.secret ) )[:-1] )

In [ ]:
class Arthur:
	def __init__( self, model ):
		self.dt = model
		self.curr_node = self.dt.root
		self.merlin = None
		self.is_done = False

	def meet( self, merlin ):
		self.merlin = merlin

	def reset( self ):
		self.curr_node = self.dt.root
		self.is_done = False

	def msg( self, response ):
		# If we are not at a leaf, lets go to the appropriate child based on the response
		if not self.curr_node.is_leaf:
			self.curr_node = self.curr_node.get_child( response )
		# If we are at a leaf, we should reqeust Merlin to terminate the round after this query
		else:
			self.is_done = True

		# Either way, get the query to be sent to Merlin
		query = self.curr_node.get_query()
		self.merlin.msg( query, self.is_done )

In [ ]:
query_max = 15
n_trials = 5

t_train = 0
m_size = 0
win = 0
query = 0

In [ ]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	model = my_fit( words )
	toc = tm.perf_counter()
	t_train += toc - tic

	with open( f"model_dump_{t}.pkl", "wb" ) as outfile:
		pickle.dump( model, outfile, protocol=pickle.HIGHEST_PROTOCOL )

	m_size += os.path.getsize( f"model_dump_{t}.pkl" )

	merlin = Merlin( query_max, words )
	arthur = Arthur( model )
	merlin.meet( arthur )
	arthur.meet( merlin )

	for ( i, secret ) in enumerate( words ):
		arthur.reset()
		merlin.reset_and_play( secret )

	win += merlin.win_count / num_words
	query += merlin.tot_query_count / num_words

In [ ]:
t_train /= n_trials
m_size /= n_trials
win /= n_trials
query /= n_trials

print( t_train, m_size, win, query )

5.119400878400006 894711.0 1.0 4.087865299012967
